# Imports

In [1]:
import pandas as pd
import numpy as np
import os

# Loading the data on NBA contracts and player statistics

In [2]:
contract_history=pd.read_csv('../raw_data/nba_contracts_history.csv')
season_17_18_1=pd.read_csv('../raw_data/2017-2018_season/nba.csv')
season_17_18_2=pd.read_csv('../raw_data/2017-2018_season/nba_extra.csv')
season_18_19_1=pd.read_csv('../raw_data/2018-2019_season/NBA_list_new.csv')
season_18_19_2=pd.read_csv('../raw_data/2018-2019_season/nbastats2018-2019.csv')
season_19_20_1=pd.read_csv('../raw_data/2019-2020_season/nba_2020_per_game.csv')
season_19_20_2=pd.read_csv('../raw_data/2019-2020_season/nba_2020_advanced.csv')

# Looking at the contract data

In [3]:
contract_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NAME            199 non-null    object 
 1   CONTRACT_START  199 non-null    int64  
 2   CONTRACT_END    199 non-null    int64  
 3   AVG_SALARY      199 non-null    float64
 4   AGE             199 non-null    float64
 5   GP              199 non-null    float64
 6   W               199 non-null    float64
 7   L               199 non-null    float64
 8   MIN             199 non-null    float64
 9   PTS             199 non-null    float64
 10  FGM             199 non-null    float64
 11  FGA             199 non-null    float64
 12  FG%             199 non-null    float64
 13  3PM             199 non-null    float64
 14  3PA             199 non-null    float64
 15  3P%             199 non-null    float64
 16  FTM             199 non-null    float64
 17  FTA             199 non-null    flo

In [4]:
contract_history = contract_history.iloc[:,0:4] # only keeping contract details
contract_history.head()

NAME  CONTRACT_START  CONTRACT_END  AVG_SALARY
0  Wesley Matthews            2019          2020   2564753.0
1      Brook Lopez            2015          2017  21165675.0
2   DeAndre Jordan            2011          2014  10759763.5
3  Markieff Morris            2015          2018   8143323.5
4    Dwight Howard            2018          2019  13410739.0

In [5]:
contract_history = contract_history[contract_history['CONTRACT_END']>=2018] # only considering contracts that ended in the 2017-2018 season or later
contract_history.head()

NAME  CONTRACT_START  CONTRACT_END  AVG_SALARY
0  Wesley Matthews            2019          2020   2564753.0
3  Markieff Morris            2015          2018   8143323.5
4    Dwight Howard            2018          2019  13410739.0
7   JaMychal Green            2019          2020   4767000.0
8     Kyle O'Quinn            2015          2018   4051312.5

In [6]:
contract_history['CONTRACT_END'].min(), contract_history['CONTRACT_END'].max()

(2018, 2020)

In [7]:
# created columns for every season with the salary the player earned that season as values
contract_history['2017-2018'] = np.select([(contract_history['CONTRACT_START']<=2017)&(contract_history['CONTRACT_END']>2017)],[contract_history['AVG_SALARY']],default=0)
contract_history['2018-2019'] = np.select([(contract_history['CONTRACT_START']<=2018)&(contract_history['CONTRACT_END']>2018)],[contract_history['AVG_SALARY']],default=0)
contract_history['2019-2020'] = np.select([(contract_history['CONTRACT_START']<=2019)&(contract_history['CONTRACT_END']>2019)],[contract_history['AVG_SALARY']],default=0)
contract_history.head()

NAME  CONTRACT_START  CONTRACT_END  AVG_SALARY  2017-2018  \
0  Wesley Matthews            2019          2020   2564753.0        0.0   
3  Markieff Morris            2015          2018   8143323.5  8143323.5   
4    Dwight Howard            2018          2019  13410739.0        0.0   
7   JaMychal Green            2019          2020   4767000.0        0.0   
8     Kyle O'Quinn            2015          2018   4051312.5  4051312.5   

    2018-2019  2019-2020  
0         0.0  2564753.0  
3         0.0        0.0  
4  13410739.0        0.0  
7         0.0  4767000.0  
8         0.0        0.0

In [8]:
contract_history['NAME'].describe()

count            116
unique           107
top       Quinn Cook
freq               3
Name: NAME, dtype: object

This shows that there are duplicate values in the NAME column. This makes sense since even the longest contracts in this era don't last that long.

In [9]:
most_frequent_name=contract_history['NAME'].describe().top
contract_history[contract_history['NAME']==most_frequent_name]

NAME  CONTRACT_START  CONTRACT_END  AVG_SALARY  2017-2018  \
44   Quinn Cook            2017          2018    867391.5   867391.5   
135  Quinn Cook            2019          2020   3000000.0        0.0   
144  Quinn Cook            2017          2018    867391.5   867391.5   

     2018-2019  2019-2020  
44         0.0        0.0  
135        0.0  3000000.0  
144        0.0        0.0

It makes sense that there are multiple values for contracts for some players since they may sign multiple contracts over these season and they may even sign multiple contracts within the same season.

In [10]:
contract_history[contract_history['NAME'].duplicated()]

NAME  CONTRACT_START  CONTRACT_END    AVG_SALARY  \
28    JaMychal Green            2017          2018  8.300000e+06   
47     Dwight Howard            2016          2018  2.375224e+07   
56     Austin Rivers            2016          2018  1.199344e+07   
87   Wesley Matthews            2015          2018  1.769944e+07   
132      Robin Lopez            2019          2020  4.767000e+06   
135       Quinn Cook            2019          2020  3.000000e+06   
144       Quinn Cook            2017          2018  8.673915e+05   
167    Avery Bradley            2018          2019  8.383500e+06   
191     Kevin Durant            2017          2018  2.750000e+07   

        2017-2018  2018-2019  2019-2020  
28   8.300000e+06        0.0        0.0  
47   2.375224e+07        0.0        0.0  
56   1.199344e+07        0.0        0.0  
87   1.769944e+07        0.0        0.0  
132  0.000000e+00        0.0  4767000.0  
135  0.000000e+00        0.0  3000000.0  
144  8.673915e+05        0.0        0.0  
167  0.000000e+00  8383500.0        0.0  
191  2.750000e+07        0.0        0.0

To get rid of the duplicates, simply add up all the salaries that apply to each indiviual player for each season.

In [11]:
salaries = contract_history.drop(columns=['CONTRACT_START','CONTRACT_END','AVG_SALARY']) # dropping no longer necessary columns
salaries = salaries.groupby('NAME').sum().reset_index() # adding the money each player made for each season
salaries.head()

NAME    2017-2018    2018-2019   2019-2020
0       Al Horford  27800803.75  27800803.75         0.0
1  Al-Farouq Aminu   7500000.00         0.00         0.0
2       Alec Burks  10375000.00         0.00         0.0
3    Anthony Davis  24604884.50  24604884.50  24604884.5
4      Aron Baynes         0.00   5323440.00         0.0

In [12]:
salaries[salaries['NAME']==most_frequent_name]

NAME  2017-2018  2018-2019  2019-2020
81  Quinn Cook  1734783.0        0.0  3000000.0

In [13]:
salaries['NAME'].describe()

count                 107
unique                107
top       Al-Farouq Aminu
freq                    1
Name: NAME, dtype: object

There no longer any duplicated names.

In [14]:
contracts_17_18 = pd.DataFrame() # creating three different dataframes for each season
contracts_18_19 = pd.DataFrame()
contracts_19_20 = pd.DataFrame()

contracts_17_18[['NAME','SALARY']] = contract_history[['NAME','2017-2018']]
contracts_18_19[['NAME','SALARY']] = contract_history[['NAME','2018-2019']]
contracts_19_20[['NAME','SALARY']] = contract_history[['NAME','2019-2020']]

contracts_17_18['SEASON']='2017-2018'
contracts_18_19['SEASON']='2018-2019'
contracts_19_20['SEASON']='2019-2020'

contracts_17_18 = contracts_17_18[contracts_17_18['SALARY']!=0]
contracts_17_18.head()

NAME      SALARY     SEASON
3     Markieff Morris   8143323.5  2017-2018
8        Kyle O'Quinn   4051312.5  2017-2018
11  Langston Galloway   7000000.0  2017-2018
12       Solomon Hill  12500000.0  2017-2018
14        Kyle Korver   5708251.0  2017-2018

Seasons where the player did not make a salary can be removed:

In [15]:
contracts_18_19 = contracts_18_19[contracts_18_19['SALARY']!=0]
contracts_18_19.head()

NAME      SALARY     SEASON
4       Dwight Howard  13410739.0  2018-2019
11  Langston Galloway   7000000.0  2018-2019
12       Solomon Hill  12500000.0  2018-2019
14        Kyle Korver   5708251.0  2018-2019
16    Pat Connaughton   1682025.0  2018-2019

In [16]:
contracts_19_20 = contracts_19_20[contracts_19_20['SALARY']!=0]
contracts_19_20.head()

NAME      SALARY     SEASON
0            Wesley Matthews   2564753.0  2019-2020
7             JaMychal Green   4767000.0  2019-2020
21             Nicolas Batum  23217391.5  2019-2020
22             Austin Rivers   2174310.0  2019-2020
26  Kentavious Caldwell-Pope   8139282.0  2019-2020

# Looking at the player statistics for the 2017-2018 season

In [17]:
season_17_18_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664 entries, 0 to 663
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rk           664 non-null    int64  
 1   Player       664 non-null    object 
 2   Pos          664 non-null    object 
 3   Age          664 non-null    int64  
 4   Tm           664 non-null    object 
 5   G            664 non-null    int64  
 6   MP           664 non-null    int64  
 7   PER          664 non-null    float64
 8   TS%          660 non-null    float64
 9   3PAr         660 non-null    float64
 10  FTr          660 non-null    float64
 11  ORB%         664 non-null    float64
 12  DRB%         664 non-null    float64
 13  TRB%         664 non-null    float64
 14  AST%         664 non-null    float64
 15  STL%         664 non-null    float64
 16  BLK%         664 non-null    float64
 17  TOV%         660 non-null    float64
 18  USG%         664 non-null    float64
 19  Unnamed:

In [18]:
season_17_18_1 = season_17_18_1.loc[:, ~season_17_18_1.columns.str.contains('^Unnamed')]
season_17_18_1.drop(columns=['Rk','Pos','Age','Tm','MP','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','OWS','DWS','WS/48','OBPM','DBPM'], inplace=True)
season_17_18_1.head()

Player   G   PER    TS%  USG%   WS  BPM  VORP
0   Alex Abrines\abrinal01  75   9.0  0.567  12.7  2.2 -2.2  -0.1
1       Quincy Acy\acyqu01  70   8.2  0.525  14.4  1.0 -2.2  -0.1
2   Steven Adams\adamsst01  76  20.6  0.630  16.7  9.7  3.3   3.3
3    Bam Adebayo\adebaba01  69  15.7  0.570  15.9  4.2  0.2   0.8
4  Arron Afflalo\afflaar01  53   5.8  0.516  12.5  0.1 -5.8  -0.7

In [19]:
season_17_18_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664 entries, 0 to 663
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      664 non-null    int64  
 1   Player  664 non-null    object 
 2   Pos     664 non-null    object 
 3   Age     664 non-null    int64  
 4   Tm      664 non-null    object 
 5   G       664 non-null    int64  
 6   GS      664 non-null    int64  
 7   MP      664 non-null    int64  
 8   FG      664 non-null    int64  
 9   FGA     664 non-null    int64  
 10  FG%     660 non-null    float64
 11  3P      664 non-null    int64  
 12  3PA     664 non-null    int64  
 13  3P%     599 non-null    float64
 14  2P      664 non-null    int64  
 15  2PA     664 non-null    int64  
 16  2P%     646 non-null    float64
 17  eFG%    660 non-null    float64
 18  FT      664 non-null    int64  
 19  FTA     664 non-null    int64  
 20  FT%     606 non-null    float64
 21  ORB     664 non-null    int64  
 22  DR

In [20]:
season_17_18_2.drop(columns=['Rk','Pos','Age','Tm','G','GS','MP','FG','FGA','3P','3PA','2P','2PA','2P%','FT','FTA','ORB','DRB','PF','eFG%','TOV'], inplace=True)
season_17_18_2.head()

Player    FG%    3P%    FT%  TRB  AST  STL  BLK   PTS
0   Alex Abrines\abrinal01  0.395  0.380  0.848  114   28   38    8   353
1       Quincy Acy\acyqu01  0.356  0.349  0.817  257   57   33   29   411
2   Steven Adams\adamsst01  0.629  0.000  0.559  685   88   92   78  1056
3    Bam Adebayo\adebaba01  0.512  0.000  0.721  381  101   32   41   477
4  Arron Afflalo\afflaar01  0.401  0.386  0.846   66   30    4    9   179

Only pulled the statistics that are needed from both of dataframes.

This empty dataframe shows both of the dataframes' player columns for the 2017-2018 season are identical.

In [21]:
season_17_18_1[season_17_18_1['Player']!=season_17_18_2['Player']]

Empty DataFrame
Columns: [Player, G, PER, TS%, USG%, WS, BPM, VORP]
Index: []

Combining the two dataframes into one:

In [22]:
season_17_18_1.drop(columns=['Player'], inplace=True)
season_17_18 = pd.concat([season_17_18_2, season_17_18_1], axis=1) # concatenating the two dataframes next to each other
season_17_18['SEASON']='2017-2018'
season_17_18.rename(columns={'Player':'NAME', 'TRB':'REB'}, inplace=True)
season_17_18.head()

NAME    FG%    3P%    FT%  REB  AST  STL  BLK   PTS   G  \
0   Alex Abrines\abrinal01  0.395  0.380  0.848  114   28   38    8   353  75   
1       Quincy Acy\acyqu01  0.356  0.349  0.817  257   57   33   29   411  70   
2   Steven Adams\adamsst01  0.629  0.000  0.559  685   88   92   78  1056  76   
3    Bam Adebayo\adebaba01  0.512  0.000  0.721  381  101   32   41   477  69   
4  Arron Afflalo\afflaar01  0.401  0.386  0.846   66   30    4    9   179  53   

    PER    TS%  USG%   WS  BPM  VORP     SEASON  
0   9.0  0.567  12.7  2.2 -2.2  -0.1  2017-2018  
1   8.2  0.525  14.4  1.0 -2.2  -0.1  2017-2018  
2  20.6  0.630  16.7  9.7  3.3   3.3  2017-2018  
3  15.7  0.570  15.9  4.2  0.2   0.8  2017-2018  
4   5.8  0.516  12.5  0.1 -5.8  -0.7  2017-2018

Getting rid of the extra characters in the names:

In [23]:
season_17_18['NAME'] = season_17_18['NAME'].str.split("\\").str.get(0)
season_17_18.head()

NAME    FG%    3P%    FT%  REB  AST  STL  BLK   PTS   G   PER  \
0   Alex Abrines  0.395  0.380  0.848  114   28   38    8   353  75   9.0   
1     Quincy Acy  0.356  0.349  0.817  257   57   33   29   411  70   8.2   
2   Steven Adams  0.629  0.000  0.559  685   88   92   78  1056  76  20.6   
3    Bam Adebayo  0.512  0.000  0.721  381  101   32   41   477  69  15.7   
4  Arron Afflalo  0.401  0.386  0.846   66   30    4    9   179  53   5.8   

     TS%  USG%   WS  BPM  VORP     SEASON  
0  0.567  12.7  2.2 -2.2  -0.1  2017-2018  
1  0.525  14.4  1.0 -2.2  -0.1  2017-2018  
2  0.630  16.7  9.7  3.3   3.3  2017-2018  
3  0.570  15.9  4.2  0.2   0.8  2017-2018  
4  0.516  12.5  0.1 -5.8  -0.7  2017-2018

In [24]:
season_17_18['NAME'].describe()

count                 664
unique                540
top       Sean Kilpatrick
freq                    5
Name: NAME, dtype: object

This shows that there are duplicated values. This makes sense because there are some players that play for multiple teams in a single season.

In [25]:
most_frequent_17_18=season_17_18['NAME'].describe().top
season_17_18[season_17_18['NAME']==most_frequent_17_18] # shows the player that is the most duplicated within the dataframe

NAME    FG%    3P%    FT%  REB  AST  STL  BLK  PTS   G   PER  \
339  Sean Kilpatrick  0.374  0.319  0.889   87   46   13    5  330  52  11.6   
340  Sean Kilpatrick  0.287  0.256  0.947   35   15    2    1   79  16   4.9   
341  Sean Kilpatrick  0.378  0.283  0.947   25   15    5    1   93  23  11.3   
342  Sean Kilpatrick  0.389  0.429  1.000    2    3    0    0   19   4  10.5   
343  Sean Kilpatrick  0.439  0.396  0.813   25   13    6    3  139   9  17.7   

       TS%  USG%   WS   BPM  VORP     SEASON  
339  0.507  24.9  0.1  -6.1  -0.7  2017-2018  
340  0.414  26.3 -0.4 -10.2  -0.4  2017-2018  
341  0.515  21.9  0.1  -6.1  -0.2  2017-2018  
342  0.503  22.6  0.0  -7.9  -0.1  2017-2018  
343  0.574  27.1  0.4  -2.3   0.0  2017-2018

In [26]:
season_17_18[season_17_18['NAME'].duplicated()]

NAME    FG%    3P%    FT%  REB  AST  STL  BLK  PTS   G   PER  \
23         Omer Asik  0.438    NaN  0.333   37    2    1    2   18  14   4.5   
24         Omer Asik  0.333    NaN  0.000   10    1    1    2    4   4   0.2   
27      Luke Babbitt  0.476  0.441  0.773   81   26    7    5  226  37  12.0   
28      Luke Babbitt  0.234  0.244    NaN   15    5    1    2   33  13   1.3   
44   Marco Belinelli  0.410  0.372  0.927  100  103   46    3  591  52  14.1   
..               ...    ...    ...    ...  ...  ...  ...  ...  ...  ..   ...   
642    Troy Williams  0.490  0.333  0.704   59   15   19    4  128  17  16.2   
649   Brandan Wright  0.576    NaN  0.636   93   13   13   25  135  27  19.3   
650   Brandan Wright  0.667    NaN    NaN    2    0    0    1    4   1  11.3   
659     Tyler Zeller  0.546  0.385  0.667  194   28    8   21  300  42  15.3   
660     Tyler Zeller  0.590  0.000  0.895  111   19    7   14  141  24  17.1   

       TS%  USG%   WS  BPM  VORP     SEASON  
23   0.423   9.4  0.0 -8.1  -0.2  2017-2018  
24   0.311   7.4 -0.1 -8.0  -0.1  2017-2018  
27   0.636  14.8  1.1 -2.0   0.0  2017-2018  
28   0.351  15.8 -0.2 -7.9  -0.2  2017-2018  
44   0.554  21.2  1.9 -1.7   0.1  2017-2018  
..     ...   ...  ...  ...   ...        ...  
642  0.572  19.5  0.6  0.9   0.2  2017-2018  
649  0.595  14.9  1.2  1.1   0.3  2017-2018  
650  0.667   8.8  0.0 -3.2   0.0  2017-2018  
659  0.587  17.9  1.5 -2.5  -0.1  2017-2018  
660  0.622  13.9  1.4 -0.1   0.2  2017-2018  

[124 rows x 17 columns]

To get rid of the duplicates, the row with the most games played will be kept to get a better idea of their performance and the other rows will be dropped:

In [27]:
season_17_18 = season_17_18.sort_values(by='G', ascending=False)
season_17_18.drop_duplicates(subset='NAME', keep='first', inplace=True)
season_17_18[season_17_18['NAME']==most_frequent_17_18]

NAME    FG%    3P%    FT%  REB  AST  STL  BLK  PTS   G   PER  \
339  Sean Kilpatrick  0.374  0.319  0.889   87   46   13    5  330  52  11.6   

       TS%  USG%   WS  BPM  VORP     SEASON  
339  0.507  24.9  0.1 -6.1  -0.7  2017-2018

In [28]:
season_17_18['NAME'].describe()

count              540
unique             540
top       Derrick Rose
freq                 1
Name: NAME, dtype: object

There are no longer any duplicate names.

In [29]:
season_17_18.head()

NAME    FG%    3P%    FT%  REB  AST  STL  BLK   PTS   G  \
332         Tyus Jones  0.457  0.349  0.877  132  232   95    6   416  82   
569   Lance Stephenson  0.427  0.289  0.661  424  235   46   17   757  82   
516      Julius Randle  0.558  0.222  0.718  654  210   43   45  1323  82   
503       Jakob Poeltl  0.659  0.500  0.594  393   57   39  100   567  82   
485  Patrick Patterson  0.398  0.386  0.870  193   57   48   23   318  82   

      PER    TS%  USG%   WS  BPM  VORP     SEASON  
332  14.1  0.572  12.7  3.6  0.5   0.9  2017-2018  
569  12.4  0.495  21.6  1.8 -2.1   0.0  2017-2018  
516  19.9  0.606  25.3  6.6  0.9   1.6  2017-2018  
503  17.5  0.662  14.8  5.6  3.3   2.0  2017-2018  
485  10.0  0.555  10.8  2.6 -1.3   0.2  2017-2018

Now the two 2017-2018 season DataFrames, one for stats and the other for salaries, can be merged.

In [30]:
# reordering the columns
column_names = ['NAME','SEASON','PTS','REB','AST','BLK','STL','G','FG%','3P%','FT%','TS%','USG%','PER','WS','BPM','VORP']
season_17_18 = season_17_18[column_names].reset_index(drop=True)
season_17_18.head()

NAME     SEASON   PTS  REB  AST  BLK  STL   G    FG%    3P%  \
0         Tyus Jones  2017-2018   416  132  232    6   95  82  0.457  0.349   
1   Lance Stephenson  2017-2018   757  424  235   17   46  82  0.427  0.289   
2      Julius Randle  2017-2018  1323  654  210   45   43  82  0.558  0.222   
3       Jakob Poeltl  2017-2018   567  393   57  100   39  82  0.659  0.500   
4  Patrick Patterson  2017-2018   318  193   57   23   48  82  0.398  0.386   

     FT%    TS%  USG%   PER   WS  BPM  VORP  
0  0.877  0.572  12.7  14.1  3.6  0.5   0.9  
1  0.661  0.495  21.6  12.4  1.8 -2.1   0.0  
2  0.718  0.606  25.3  19.9  6.6  0.9   1.6  
3  0.594  0.662  14.8  17.5  5.6  3.3   2.0  
4  0.870  0.555  10.8  10.0  2.6 -1.3   0.2

In [31]:
season_17_18 = pd.merge(season_17_18, contracts_17_18, on=['NAME','SEASON'])
season_17_18.head()

NAME     SEASON   PTS  REB  AST  BLK  STL   G    FG%    3P%  \
0  Patrick Patterson  2017-2018   318  193   57   23   48  82  0.398  0.386   
1      E'Twaun Moore  2017-2018  1022  238  187   12   79  82  0.508  0.425   
2    Bismack Biyombo  2017-2018   468  468   66   95   21  82  0.520  0.000   
3    Khris Middleton  2017-2018  1652  429  328   21  119  82  0.466  0.359   
4        Cory Joseph  2017-2018   649  263  260   18   80  82  0.424  0.353   

     FT%    TS%  USG%   PER   WS  BPM  VORP        SALARY  
0  0.870  0.555  10.8  10.0  2.6 -1.3   0.2  4.325064e+06  
1  0.706  0.593  16.1  12.1  4.4 -0.2   1.2  8.680000e+06  
2  0.650  0.556  14.9  14.1  2.9 -1.6   0.2  1.700000e+07  
3  0.884  0.577  24.6  17.4  6.9  0.4   1.8  1.752069e+07  
4  0.745  0.503  14.8  11.0  3.1 -1.2   0.5  7.472500e+06

# Looking at the player statistics for the 2018-2019 season

In [32]:
season_18_19_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER        530 non-null    object 
 1   TEAM          530 non-null    object 
 2   AGE           530 non-null    int64  
 3   HEIGHT        530 non-null    int64  
 4   WEIGHT        530 non-null    int64  
 5   COLLEGE       530 non-null    object 
 6   COUNTRY       530 non-null    object 
 7   DRAFT YEAR    530 non-null    object 
 8   DRAFT ROUND   530 non-null    object 
 9   DRAFT NUMBER  530 non-null    object 
 10  GP            530 non-null    int64  
 11  PTS           530 non-null    float64
 12  REB           530 non-null    float64
 13  AST           530 non-null    float64
 14  NETRTG        530 non-null    float64
 15  OREB%         530 non-null    object 
 16  DREB%         530 non-null    object 
 17  USG%          530 non-null    object 
 18  TS%           530 non-null    

In [33]:
season_18_19_1.drop(columns=['TEAM','AGE','HEIGHT','WEIGHT','COLLEGE','COUNTRY','DRAFT YEAR','DRAFT ROUND','DRAFT NUMBER','NETRTG','OREB%','DREB%','AST%','COUN_CODE'], inplace=True)
season_18_19_1.head()

PLAYER  GP   PTS  REB  AST    USG%     TS%
0     Aaron Gordon  78  16.0  7.4  3.7  21.20%  53.80%
1    Aaron Holiday  50   5.9  1.3  1.7  20.40%  51.80%
2      Abdel Nader  61   4.0  1.9  0.3  14.70%  52.20%
3       Al Horford  68  13.6  6.7  4.2  18.70%  60.50%
4  Al-Farouq Aminu  81   9.4  7.5  1.3  13.30%  56.80%

In [34]:
season_18_19_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      521 non-null    object 
 1   Height    521 non-null    int64  
 2   Weight    521 non-null    int64  
 3   Team      470 non-null    object 
 4   Age       521 non-null    int64  
 5   Salary    521 non-null    object 
 6   Points    521 non-null    float64
 7   Blocks    521 non-null    float64
 8   Steals    521 non-null    float64
 9   Assists   521 non-null    float64
 10  Rebounds  521 non-null    float64
 11  FT%       499 non-null    float64
 12  FTA       521 non-null    float64
 13  FG3%      489 non-null    float64
 14  FG3A      521 non-null    float64
 15  FG%       519 non-null    float64
 16  FGA       521 non-null    float64
 17  MP        521 non-null    float64
 18  G         521 non-null    int64  
 19  PER       521 non-null    float64
 20  OWS       521 non-null    float6

In [35]:
season_18_19_2.drop(columns=['Height','Weight','Team','Age','Salary','FTA','FG3A','FGA','MP','OWS','DWS','WS48','Points','Assists','Rebounds','USG'], inplace=True)
season_18_19_2.head()

Name  Blocks  Steals    FT%   FG3%    FG%   G   PER   WS  BPM  VORP
0  Alex Abrines     0.2     0.5  0.923  0.323  0.357  31   6.3  0.6 -3.4  -0.2
1    Quincy Acy     0.4     0.1  0.700  0.133  0.222  10   2.9 -0.1 -5.9  -0.1
2  Jaylen Adams     0.1     0.4  0.778  0.338  0.345  34   7.6  0.1 -4.4  -0.3
3  Steven Adams     1.0     1.5  0.500  0.000  0.595  80  18.5  9.1  2.7   3.2
4   Bam Adebayo     0.8     0.9  0.735  0.200  0.576  82  17.9  6.8  3.0   2.4

Only pulled the statistics that are needed.

The above information showed that the dataframes have different amounts of rows which means that they have different amounts of players. Before the two dataframes are combined, the names on both have to be reconciled. Before that, duplicate names in each dataframe have to be removed if there are any.

In [36]:
season_18_19_1['PLAYER'].describe()

count                 530
unique                530
top       Al-Farouq Aminu
freq                    1
Name: PLAYER, dtype: object

In [37]:
season_18_19_2['Name'].describe()

count              521
unique             521
top       Derrick Rose
freq                 1
Name: Name, dtype: object

The above shows there are no duplicates in either dataframe. So now both dataframes will be reconciled using the columns that contain the player names.

In [38]:
#season_18_19_1.set_index('PLAYER', inplace=True)
#season_18_19_2.set_index('Name', inplace=True)
#season_18_19 = pd.concat([season_18_19_1, season_18_19_2], axis=1, join='inner') # dataframes are concatenated next to each other with an inner join

season_18_19_1 = season_18_19_1.rename(columns={'PLAYER':'NAME'})
season_18_19_2 = season_18_19_2.rename(columns={'Name':'NAME'})
season_18_19 = pd.merge(season_18_19_1, season_18_19_2, on='NAME') # the two 2018-2019 stats DataFrames become merged

season_18_19.head()

NAME  GP   PTS  REB  AST    USG%     TS%  Blocks  Steals    FT%  \
0     Aaron Gordon  78  16.0  7.4  3.7  21.20%  53.80%     0.7     0.7  0.731   
1    Aaron Holiday  50   5.9  1.3  1.7  20.40%  51.80%     0.3     0.4  0.820   
2      Abdel Nader  61   4.0  1.9  0.3  14.70%  52.20%     0.2     0.3  0.750   
3       Al Horford  68  13.6  6.7  4.2  18.70%  60.50%     1.3     0.9  0.821   
4  Al-Farouq Aminu  81   9.4  7.5  1.3  13.30%  56.80%     0.4     0.8  0.867   

    FG3%    FG%   G   PER   WS  BPM  VORP  
0  0.349  0.449  78  15.1  5.1  1.0   2.0  
1  0.339  0.401  50  11.9  0.9 -2.6  -0.1  
2  0.320  0.423  61   8.8  0.9 -5.1  -0.5  
3  0.360  0.535  68  20.2  7.5  4.8   3.4  
4  0.343  0.433  81  13.2  5.8  0.9   1.7

Now to adjust the columns so that the statistics for the 2018-2019 season dataframe match that of the 2017-2018 season dataframe.

In [39]:
season_18_19['SEASON']='2018-2019'
season_18_19.drop(columns=['GP'], inplace=True)
season_18_19.rename(columns={'Blocks':'BLK', 'Steals':'STL', 'FG3%':'3P%'}, inplace=True)
season_18_19['PTS']=season_18_19['PTS']*season_18_19['G'] # getting total points instead of points per game
season_18_19['REB']=season_18_19['REB']*season_18_19['G']
season_18_19['AST']=season_18_19['AST']*season_18_19['G']
season_18_19['BLK']=season_18_19['BLK']*season_18_19['G']
season_18_19['STL']=season_18_19['STL']*season_18_19['G']
season_18_19['USG%'] = pd.to_numeric(season_18_19['USG%'].str.split("%").str.get(0)) # remvoving '%' from string and then converting it to a numerical data type
season_18_19['TS%'] = pd.to_numeric(season_18_19['TS%'].str.split("%").str.get(0))/100
season_18_19 = season_18_19[column_names].reset_index(drop=True) # reordering the columns
season_18_19.head()

NAME     SEASON     PTS    REB    AST   BLK   STL   G    FG%  \
0     Aaron Gordon  2018-2019  1248.0  577.2  288.6  54.6  54.6  78  0.449   
1    Aaron Holiday  2018-2019   295.0   65.0   85.0  15.0  20.0  50  0.401   
2      Abdel Nader  2018-2019   244.0  115.9   18.3  12.2  18.3  61  0.423   
3       Al Horford  2018-2019   924.8  455.6  285.6  88.4  61.2  68  0.535   
4  Al-Farouq Aminu  2018-2019   761.4  607.5  105.3  32.4  64.8  81  0.433   

     3P%    FT%    TS%  USG%   PER   WS  BPM  VORP  
0  0.349  0.731  0.538  21.2  15.1  5.1  1.0   2.0  
1  0.339  0.820  0.518  20.4  11.9  0.9 -2.6  -0.1  
2  0.320  0.750  0.522  14.7   8.8  0.9 -5.1  -0.5  
3  0.360  0.821  0.605  18.7  20.2  7.5  4.8   3.4  
4  0.343  0.867  0.568  13.3  13.2  5.8  0.9   1.7

Now the salary DataFrame and the stats DataFrame for the 2018-2019 season can be merged.

In [40]:
season_18_19 = pd.merge(season_18_19, contracts_18_19, on=['NAME','SEASON'])
season_18_19.head()

NAME     SEASON     PTS    REB    AST    BLK   STL   G    FG%  \
0       Al Horford  2018-2019   924.8  455.6  285.6   88.4  61.2  68  0.535   
1    Anthony Davis  2018-2019  1450.4  672.0  218.4  134.4  89.6  56  0.517   
2      Aron Baynes  2018-2019   285.6  239.7   56.1   35.7  10.2  51  0.471   
3    Avery Bradley  2018-2019   623.7  176.4  151.2   18.9  44.1  63  0.408   
4  Bismack Biyombo  2018-2019   237.6  248.4   32.4   43.2  10.8  54  0.571   

     3P%    FT%    TS%  USG%   PER   WS  BPM  VORP       SALARY  
0  0.360  0.821  0.605  18.7  20.2  7.5  4.8   3.4  27800803.75  
1  0.331  0.794  0.597  28.7  30.3  9.5  8.5   4.9  24604884.50  
2  0.344  0.855  0.567  14.5  14.0  2.4  0.2   0.5   5323440.00  
3  0.351  0.860  0.496  15.9   7.6  0.3 -2.8  -0.4   8383500.00  
4    NaN  0.637  0.602  12.6  14.4  2.0 -1.1   0.2  17000000.00

# Looking at the player statistics for the 2019-2020 season

In [41]:
season_19_20_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651 entries, 0 to 650
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  651 non-null    object 
 1   Pos     651 non-null    object 
 2   Age     651 non-null    int64  
 3   Tm      651 non-null    object 
 4   G       651 non-null    int64  
 5   GS      651 non-null    int64  
 6   MP      651 non-null    float64
 7   FG      651 non-null    float64
 8   FGA     651 non-null    float64
 9   FG%     649 non-null    float64
 10  3P      651 non-null    float64
 11  3PA     651 non-null    float64
 12  3P%     616 non-null    float64
 13  2P      651 non-null    float64
 14  2PA     651 non-null    float64
 15  2P%     645 non-null    float64
 16  eFG%    649 non-null    float64
 17  FT      651 non-null    float64
 18  FTA     651 non-null    float64
 19  FT%     618 non-null    float64
 20  ORB     651 non-null    float64
 21  DRB     651 non-null    float64
 22  TR

In [42]:
season_19_20_1.drop(columns=['Pos','Age','Tm','GS','MP','FG','FGA','3P','3PA','2P','2PA','2P%','eFG%','FT','FTA','ORB','DRB','TOV','PF'], inplace=True)
season_19_20_1.head()

Player   G    FG%    3P%    FT%   TRB  AST  STL  BLK  \
0              Steven Adams  63  0.592  0.333  0.582   9.3  2.3  0.8  1.1   
1               Bam Adebayo  72  0.557  0.143  0.691  10.2  5.1  1.1  1.3   
2         LaMarcus Aldridge  53  0.493  0.389  0.827   7.4  2.4  0.7  1.6   
3            Kyle Alexander   2  0.500    NaN    NaN   1.5  0.0  0.0  0.0   
4  Nickeil Alexander-Walker  47  0.368  0.346  0.676   1.8  1.9  0.4  0.2   

    PTS  
0  10.9  
1  15.9  
2  18.9  
3   1.0  
4   5.7

In [43]:
season_19_20_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651 entries, 0 to 650
Data columns (total 26 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  651 non-null    object 
 1   Pos     651 non-null    object 
 2   Age     651 non-null    int64  
 3   Tm      651 non-null    object 
 4   G       651 non-null    int64  
 5   MP      651 non-null    int64  
 6   PER     651 non-null    float64
 7   TS%     649 non-null    float64
 8   3PAr    649 non-null    float64
 9   FTr     649 non-null    float64
 10  ORB%    651 non-null    float64
 11  DRB%    651 non-null    float64
 12  TRB%    651 non-null    float64
 13  AST%    651 non-null    float64
 14  STL%    651 non-null    float64
 15  BLK%    651 non-null    float64
 16  TOV%    649 non-null    float64
 17  USG%    651 non-null    float64
 18  OWS     651 non-null    float64
 19  DWS     651 non-null    float64
 20  WS      651 non-null    float64
 21  WS/48   651 non-null    float64
 22  OB

In [44]:
season_19_20_2.drop(columns=['Pos','Age','Tm','G','MP','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','OWS','DWS','WS/48','OBPM','DBPM'], inplace=True)
season_19_20_2.head()

Player   PER    TS%  USG%   WS  BPM  VORP
0              Steven Adams  20.5  0.604  17.3  6.5  2.9   2.1
1               Bam Adebayo  20.3  0.598  21.2  8.5  3.4   3.3
2         LaMarcus Aldridge  19.7  0.571  23.4  4.5  1.4   1.5
3            Kyle Alexander   4.7  0.500  10.2  0.0 -9.6   0.0
4  Nickeil Alexander-Walker   8.9  0.473  23.3 -0.2 -4.6  -0.4

Only pulled the statistics that are needed from both of dataframes.

This empty dataframe shows both of the dataframes' player columns for the 2017-2018 season are identical.

In [45]:
season_19_20_1[season_19_20_1['Player']!=season_19_20_2['Player']]

Empty DataFrame
Columns: [Player, G, FG%, 3P%, FT%, TRB, AST, STL, BLK, PTS]
Index: []

Combining the two dataframes into one:

In [46]:
season_19_20_2.drop(columns=['Player'], inplace=True)
season_19_20 = pd.concat([season_19_20_1, season_19_20_2], axis=1) # concatenating the two dataframes next to each other
season_19_20['SEASON']='2019-2020'
season_19_20.rename(columns={'Player':'NAME', 'TRB':'REB'}, inplace=True)
season_19_20.head()

NAME   G    FG%    3P%    FT%   REB  AST  STL  BLK  \
0              Steven Adams  63  0.592  0.333  0.582   9.3  2.3  0.8  1.1   
1               Bam Adebayo  72  0.557  0.143  0.691  10.2  5.1  1.1  1.3   
2         LaMarcus Aldridge  53  0.493  0.389  0.827   7.4  2.4  0.7  1.6   
3            Kyle Alexander   2  0.500    NaN    NaN   1.5  0.0  0.0  0.0   
4  Nickeil Alexander-Walker  47  0.368  0.346  0.676   1.8  1.9  0.4  0.2   

    PTS   PER    TS%  USG%   WS  BPM  VORP     SEASON  
0  10.9  20.5  0.604  17.3  6.5  2.9   2.1  2019-2020  
1  15.9  20.3  0.598  21.2  8.5  3.4   3.3  2019-2020  
2  18.9  19.7  0.571  23.4  4.5  1.4   1.5  2019-2020  
3   1.0   4.7  0.500  10.2  0.0 -9.6   0.0  2019-2020  
4   5.7   8.9  0.473  23.3 -0.2 -4.6  -0.4  2019-2020

In [47]:
season_19_20['NAME'].describe()

count                  651
unique                 529
top       Anthony Tolliver
freq                     4
Name: NAME, dtype: object

This shows that there are duplicated values. This makes sense because there are some players that play for multiple teams in a single season.

In [48]:
most_frequent_19_20=season_19_20['NAME'].describe().top
season_19_20[season_19_20['NAME']==most_frequent_19_20] # shows the player that is the most duplicated within the dataframe

NAME   G    FG%    3P%    FT%  REB  AST  STL  BLK  PTS  PER  \
576  Anthony Tolliver  55  0.357  0.338  0.724  2.8  0.8  0.3  0.2  3.6  7.0   
577  Anthony Tolliver  33  0.368  0.337  0.684  3.3  0.9  0.2  0.3  3.9  7.5   
578  Anthony Tolliver   9  0.176  0.133  0.500  1.2  0.3  0.4  0.1  1.0  0.2   
579  Anthony Tolliver  13  0.396  0.415  0.875  2.5  0.8  0.5  0.1  4.8  8.0   

       TS%  USG%   WS  BPM  VORP     SEASON  
576  0.513  11.1  0.5 -3.2  -0.3  2019-2020  
577  0.515  11.2  0.4 -3.3  -0.2  2019-2020  
578  0.252  11.6 -0.2 -8.8  -0.1  2019-2020  
579  0.602  10.7  0.3 -1.2   0.0  2019-2020

In [49]:
season_19_20[season_19_20['NAME'].duplicated()]

NAME   G    FG%    3P%    FT%  REB  AST  STL  BLK   PTS  \
19       Trevor Ariza  32  0.388  0.352  0.778  4.6  1.6  1.1  0.2   6.0   
20       Trevor Ariza  21  0.491  0.400  0.872  4.8  2.0  1.6  0.4  11.0   
32   Keita Bates-Diop  37  0.422  0.330  0.708  3.0  0.8  0.5  0.5   6.8   
33   Keita Bates-Diop   7  0.464  0.333  0.800  2.4  0.0  0.3  0.6   5.3   
37      Kent Bazemore  43  0.347  0.327  0.806  4.0  1.4  1.0  0.7   7.9   
..                ...  ..    ...    ...    ...  ...  ...  ...  ...   ...   
621       Paul Watson   8  0.526  0.444  0.778  1.9  0.6  0.4  0.1   3.9   
628    Andrew Wiggins  42  0.444  0.331  0.720  5.2  3.7  0.7  0.9  22.4   
629    Andrew Wiggins  12  0.457  0.339  0.672  4.6  3.6  1.3  1.4  19.4   
635   Marvin Williams  41  0.448  0.376  0.860  2.7  1.0  0.6  0.5   6.7   
636   Marvin Williams  17  0.439  0.308  0.857  4.4  1.1  0.6  0.5   4.0   

      PER    TS%  USG%   WS  BPM  VORP     SEASON  
19    9.4  0.543  11.2  1.1 -0.7   0.3  2019-2020  
20   12.8  0.643  12.5  1.4 -0.4   0.3  2019-2020  
32   12.5  0.535  15.8  1.0 -0.9   0.2  2019-2020  
33   12.1  0.571  15.7  0.2 -4.9  -0.1  2019-2020  
37    7.5  0.475  15.9 -0.2 -3.2  -0.3  2019-2020  
..    ...    ...   ...  ...  ...   ...        ...  
621  17.3  0.675  15.9  0.3  2.9   0.1  2019-2020  
628  16.1  0.535  28.0  1.4 -0.4   0.6  2019-2020  
629  17.5  0.542  25.4  0.5 -0.2   0.2  2019-2020  
635  11.8  0.593  14.0  1.5 -1.3   0.1  2019-2020  
636   9.8  0.566   8.7  0.9  0.9   0.2  2019-2020  

[122 rows x 17 columns]

To get rid of the duplicates, the row with the most games played will be kept to get a better idea of their performance and the other rows will be dropped:

In [50]:
season_19_20 = season_19_20.sort_values(by='G', ascending=False)
season_19_20.drop_duplicates(subset='NAME', keep='first', inplace=True)
season_19_20[season_19_20['NAME']==most_frequent_19_20]

NAME   G    FG%    3P%    FT%  REB  AST  STL  BLK  PTS  PER  \
576  Anthony Tolliver  55  0.357  0.338  0.724  2.8  0.8  0.3  0.2  3.6  7.0   

       TS%  USG%   WS  BPM  VORP     SEASON  
576  0.513  11.1  0.5 -3.2  -0.3  2019-2020

In [51]:
season_19_20['NAME'].describe()

count              529
unique             529
top       Derrick Rose
freq                 1
Name: NAME, dtype: object

There are no longer any duplicate names.

Now to adjust the columns so that the statistics for the 2019-2020 season dataframe match that of the 2017-2018 season dataframe.

In [52]:
season_19_20['PTS']=season_19_20['PTS']*season_19_20['G'] # getting total points instead of points per game
season_19_20['REB']=season_19_20['REB']*season_19_20['G']
season_19_20['AST']=season_19_20['AST']*season_19_20['G']
season_19_20['BLK']=season_19_20['BLK']*season_19_20['G']
season_19_20['STL']=season_19_20['STL']*season_19_20['G']
season_19_20 = season_19_20[column_names].reset_index(drop=True) # reordering the columns
season_19_20.head()

NAME     SEASON     PTS    REB    AST   BLK   STL   G    FG%  \
0      Maxi Kleber  2019-2020   673.4  384.8   88.8  81.4  22.2  74  0.461   
1     Monte Morris  2019-2020   657.0  138.7  255.5  14.6  58.4  73  0.459   
2     Delon Wright  2019-2020   503.7  277.4  240.9  21.9  87.6  73  0.462   
3     Nikola Jokić  2019-2020  1452.7  708.1  511.0  43.8  87.6  73  0.528   
4  Duncan Robinson  2019-2020   985.5  233.6  102.2  21.9  36.5  73  0.470   

     3P%    FT%    TS%  USG%   PER   WS  BPM  VORP  
0  0.373  0.849  0.605  13.9  13.6  4.8  0.4   1.1  
1  0.378  0.843  0.545  17.5  14.9  3.8 -0.2   0.8  
2  0.370  0.770  0.554  14.4  15.4  4.4  1.9   1.5  
3  0.314  0.817  0.605  26.6  24.9  9.8  7.4   5.5  
4  0.446  0.931  0.684  16.0  13.1  5.3  0.5   1.3

Now to merge the 2019-2020 season stats DataFrame with the salary DataFrame.

In [53]:
season_19_20 = pd.merge(season_19_20, contracts_19_20, on=['NAME','SEASON'])
season_19_20.head()

NAME     SEASON    PTS    REB    AST   BLK   STL   G  \
0              Jerami Grant  2019-2020  852.0  248.5   85.2  56.8  49.7  71   
1  Kentavious Caldwell-Pope  2019-2020  641.7  144.9  110.4  13.8  55.2  69   
2             Austin Rivers  2019-2020  598.4  176.8  115.6   6.8  47.6  68   
3           Wesley Matthews  2019-2020  495.8  167.5   93.8   6.7  40.2  67   
4               Robin Lopez  2019-2020  356.4  158.4   46.2  46.2  13.2  66   

     FG%    3P%    FT%    TS%  USG%   PER   WS  BPM  VORP     SALARY  
0  0.478  0.389  0.750  0.591  18.0  14.4  4.4 -0.5   0.7  8839743.0  
1  0.467  0.385  0.775  0.584  14.7  11.0  3.7 -0.8   0.5  8139282.0  
2  0.421  0.356  0.703  0.542  15.6  10.4  2.4 -2.5  -0.2  2174310.0  
3  0.396  0.364  0.765  0.547  12.5   8.1  3.2 -1.8   0.1  2564753.0  
4  0.492  0.333  0.528  0.554  16.6  10.2  1.5 -2.6  -0.2  4767000.0

# Now to combine all three DataFrames into one:

In [54]:
nba_data = pd.concat([season_17_18, season_18_19, season_19_20])
nba_data.head()

NAME     SEASON     PTS    REB    AST   BLK    STL   G    FG%  \
0  Patrick Patterson  2017-2018   318.0  193.0   57.0  23.0   48.0  82  0.398   
1      E'Twaun Moore  2017-2018  1022.0  238.0  187.0  12.0   79.0  82  0.508   
2    Bismack Biyombo  2017-2018   468.0  468.0   66.0  95.0   21.0  82  0.520   
3    Khris Middleton  2017-2018  1652.0  429.0  328.0  21.0  119.0  82  0.466   
4        Cory Joseph  2017-2018   649.0  263.0  260.0  18.0   80.0  82  0.424   

     3P%    FT%    TS%  USG%   PER   WS  BPM  VORP        SALARY  
0  0.386  0.870  0.555  10.8  10.0  2.6 -1.3   0.2  4.325064e+06  
1  0.425  0.706  0.593  16.1  12.1  4.4 -0.2   1.2  8.680000e+06  
2  0.000  0.650  0.556  14.9  14.1  2.9 -1.6   0.2  1.700000e+07  
3  0.359  0.884  0.577  24.6  17.4  6.9  0.4   1.8  1.752069e+07  
4  0.353  0.745  0.503  14.8  11.0  3.1 -1.2   0.5  7.472500e+06

In [55]:
nba_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 14
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NAME    159 non-null    object 
 1   SEASON  159 non-null    object 
 2   PTS     159 non-null    float64
 3   REB     159 non-null    float64
 4   AST     159 non-null    float64
 5   BLK     159 non-null    float64
 6   STL     159 non-null    float64
 7   G       159 non-null    int64  
 8   FG%     159 non-null    float64
 9   3P%     151 non-null    float64
 10  FT%     158 non-null    float64
 11  TS%     159 non-null    float64
 12  USG%    159 non-null    float64
 13  PER     159 non-null    float64
 14  WS      159 non-null    float64
 15  BPM     159 non-null    float64
 16  VORP    159 non-null    float64
 17  SALARY  159 non-null    float64
dtypes: float64(15), int64(1), object(2)
memory usage: 23.6+ KB


This shows that there are NaN values within the 3-point-percentage (3P%) and free-throw-percentage (FT%) columns. This could complicate working with these columns so I'll check them out.

In [56]:
nba_data[np.isnan(nba_data['3P%'])]

NAME     SEASON    PTS     REB    AST   BLK   STL   G    FG%  \
23    DeAndre Jordan  2017-2018  927.0  1171.0  117.0  71.0  39.0  77  0.645   
65  Tristan Thompson  2017-2018  307.0   352.0   33.0  17.0  16.0  53  0.562   
67    Tyson Chandler  2017-2018  300.0   418.0   53.0  27.0  16.0  46  0.647   
70  Boban Marjanovic  2017-2018  235.0   144.0   22.0  11.0  10.0  39  0.534   
4    Bismack Biyombo  2018-2019  237.6   248.4   32.4  43.2  10.8  54  0.571   
10     Dwight Howard  2018-2019  115.2    82.8    3.6   3.6   7.2   9  0.623   
45      Nerlens Noel  2018-2019  377.3   323.4   46.2  92.4  69.3  77  0.587   
57  Tristan Thompson  2018-2019  468.7   438.6   86.0  17.2  30.1  43  0.529   

    3P%    FT%    TS%  USG%   PER   WS  BPM  VORP       SALARY  
23  NaN  0.580  0.648  15.2  20.2  9.4  2.1   2.5  21598556.25  
65  NaN  0.544  0.569  12.6  13.6  2.5 -2.5  -0.1  16399826.00  
67  NaN  0.624  0.658  10.6  14.2  2.7 -0.3   0.5  13000000.00  
70  NaN  0.794  0.616  29.4  26.0  1.5 -0.6   0.1   7000000.00  
4   NaN  0.637  0.602  12.6  14.4  2.0 -1.1   0.2  17000000.00  
10  NaN  0.604  0.638  19.2  17.4  0.6 -5.2  -0.2  13410739.00  
45  NaN  0.684  0.608  13.6  19.3  4.3  3.9   1.6   1820148.00  
57  NaN  0.642  0.551  17.3  18.4  2.6  0.3   0.7  16399826.00

So for the players associated with the NaNs in the 3P% columns, the first thing I noticed was that they all played the same position: center. Additionally, they all play a more tradtitional roll for centers offensively, that is, staying close to the basket and scoring from there. What I am saying is that these players are not know for shootong 3-pointers. In fact, the reason why there is a NaN under the 3P% column for these players is because they did not take any 3-pointers. In order to simplify using this data I will replace the NaNs with 0s.

In [57]:
nba_data['3P%'].fillna(0, inplace=True)
nba_data[np.isnan(nba_data['3P%'])]

Empty DataFrame
Columns: [NAME, SEASON, PTS, REB, AST, BLK, STL, G, FG%, 3P%, FT%, TS%, USG%, PER, WS, BPM, VORP, SALARY]
Index: []

In [58]:
nba_data[np.isnan(nba_data['FT%'])]

NAME     SEASON  PTS  REB  AST  BLK  STL  G  FG%  3P%  FT%  TS%  \
84  Gordon Hayward  2017-2018  2.0  1.0  0.0  0.0  0.0  1  0.5  0.0  NaN  0.5   

    USG%  PER   WS   BPM  VORP      SALARY  
84  17.9  7.3  0.0 -12.0   0.0  31214295.0

The reason why Gordon Hayward has an NaN under his FT% columns is because he did not take any free throws that season. Normally this would be very strange but it also shows, under the games played column (G), that he only played 1 game that season. During this season, in the very first game og the season Gordon Hayward suffered a season-ending injury. Very unfortunate considering his very large salary of over \\$31,000,000. To prevent this data from skewing the results, I will remove this row from the DataFrame.

In [59]:
nba_data.drop(nba_data[np.isnan(nba_data['FT%'])].index, inplace=True)
nba_data[np.isnan(nba_data['FT%'])]

Empty DataFrame
Columns: [NAME, SEASON, PTS, REB, AST, BLK, STL, G, FG%, 3P%, FT%, TS%, USG%, PER, WS, BPM, VORP, SALARY]
Index: []

In [60]:
nba_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 14
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NAME    158 non-null    object 
 1   SEASON  158 non-null    object 
 2   PTS     158 non-null    float64
 3   REB     158 non-null    float64
 4   AST     158 non-null    float64
 5   BLK     158 non-null    float64
 6   STL     158 non-null    float64
 7   G       158 non-null    int64  
 8   FG%     158 non-null    float64
 9   3P%     158 non-null    float64
 10  FT%     158 non-null    float64
 11  TS%     158 non-null    float64
 12  USG%    158 non-null    float64
 13  PER     158 non-null    float64
 14  WS      158 non-null    float64
 15  BPM     158 non-null    float64
 16  VORP    158 non-null    float64
 17  SALARY  158 non-null    float64
dtypes: float64(15), int64(1), object(2)
memory usage: 23.5+ KB


# Save the DataFrame into a csv file

In [61]:
nba_data.to_csv(r'..\data\nba_data.csv')